In [1]:
!wget http://files.grouplens.org/datasets/movielens/ml-10m.zip

--2018-12-09 21:51:13--  http://files.grouplens.org/datasets/movielens/ml-10m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.34.235
Connecting to files.grouplens.org (files.grouplens.org)|128.101.34.235|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65566137 (63M) [application/zip]
Saving to: ‘ml-10m.zip’

ml-10m.zip          100%[===================>]  62.53M  50.8MB/s    in 1.2s    

2018-12-09 21:51:15 (50.8 MB/s) - ‘ml-10m.zip’ saved [65566137/65566137]



In [2]:
!unzip ml-10m.zip

Archive:  ml-10m.zip
   creating: ml-10M100K/
  inflating: ml-10M100K/allbut.pl    
  inflating: ml-10M100K/movies.dat   
  inflating: ml-10M100K/ratings.dat  
  inflating: ml-10M100K/README.html  
  inflating: ml-10M100K/split_ratings.sh  
  inflating: ml-10M100K/tags.dat     


In [3]:
# Importing tensorflow
import tensorflow as tf
# Importing some more libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/font_manager.py:278: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [4]:
# reading the ratings data
ratings = pd.read_csv('ml-10M100K/ratings.dat',\
          sep="::", header = None, engine='python')
# pivot the data to get it at a user level
ratings_pivot = ratings[[0,1,2]].pivot(values=2, index=0, columns=1 ).fillna(0)
# creating train and test sets
X_train, X_test = train_test_split(ratings_pivot, train_size=0.8)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [14]:
X_train.shape

(55902, 10677)

In [9]:
X_test.shape

(13976, 10677)

In [16]:
# Deciding how many nodes each layer should have
n_nodes_inpl = 10677  
n_nodes_hl1  = 256  
n_nodes_outl = 10677  
# first hidden layer has 784*32 weights and 32 biases
hidden_1_layer_vals = {'weights':tf.Variable(tf.random_normal([n_nodes_inpl+1,n_nodes_hl1]))}
output_layer_vals = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1+1,n_nodes_outl])) }

In [17]:
# user with 3706 ratings goes in
input_layer = tf.placeholder('float', [None, 10677])
# add a constant node to the first layer
# it needs to have the same shape as the input layer for me to be
# able to concatinate it later
input_layer_const = tf.fill( [tf.shape(input_layer)[0], 1] ,1.0  )
input_layer_concat =  tf.concat([input_layer, input_layer_const], 1)
# multiply output of input_layer wth a weight matrix 
layer_1 = tf.nn.sigmoid(tf.matmul(input_layer_concat,\
hidden_1_layer_vals['weights']))
# adding one bias node to the hidden layer
layer1_const = tf.fill( [tf.shape(layer_1)[0], 1] ,1.0  )
layer_concat =  tf.concat([layer_1, layer1_const], 1)
# multiply output of hidden with a weight matrix to get final output
output_layer = tf.matmul( layer_concat,output_layer_vals['weights'])
# output_true shall have the original shape for error calculations
output_true = tf.placeholder('float', [None, 10677])
# define our cost function
meansq =    tf.reduce_mean(tf.square(output_layer - output_true))
# define our optimizer
learn_rate = 0.1   # how fast the model should learn
optimizer = tf.train.AdagradOptimizer(learn_rate).minimize(meansq)

In [18]:
# initialising variables and starting the session
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
# defining batch size, number of epochs and learning rate
batch_size = 256  # how many images to use together for training
hm_epochs =10    # how many times to go through the entire dataset
tot_images = X_train.shape[0] # total number of images

In [19]:
# running the model for a 10 epochs taking 256 users in batches
# total improvement is printed out after each epoch
def execute():
    for epoch in range(hm_epochs):
        epoch_loss = 0    # initializing error as 0

        for i in range(int(tot_images/batch_size)):
            epoch_x = X_train[ i*batch_size : (i+1)*batch_size ]
            _, c = sess.run([optimizer, meansq],\
                   feed_dict={input_layer: epoch_x, \
                   output_true: epoch_x})
            epoch_loss += c

        output_train = sess.run(output_layer,\
                   feed_dict={input_layer:X_train})
        output_test = sess.run(output_layer,\
                       feed_dict={input_layer:X_test})

        print('MSE train', MSE(output_train, X_train),'MSE test', MSE(output_test, X_test))      
        print('Epoch', epoch, '/', hm_epochs, 'loss:',epoch_loss)

In [20]:
execute()

MSE train 21.02667199444175 MSE test 21.269128203290155
Epoch 0 / 10 loss: 10158.817810058594
MSE train 12.540689016754229 MSE test 12.79793284500949
Epoch 1 / 10 loss: 3471.212664604187
MSE train 9.449818040178554 MSE test 9.690455625060112
Epoch 2 / 10 loss: 2364.2586345672607
MSE train 7.78168182759592 MSE test 8.007625569608386
Epoch 3 / 10 loss: 1869.1128058433533
MSE train 6.71481371102814 MSE test 6.928391925030984
Epoch 4 / 10 loss: 1578.2136611938477
MSE train 5.957622213273727 MSE test 6.162226510151415
Epoch 5 / 10 loss: 1381.688039779663
MSE train 5.386035596716523 MSE test 5.589230112988447
Epoch 6 / 10 loss: 1237.6742644309998
MSE train 4.937154088723258 MSE test 5.141065937009644
Epoch 7 / 10 loss: 1126.9541451931
MSE train 4.57182836938838 MSE test 4.774583705218716
Epoch 8 / 10 loss: 1038.2290863990784
MSE train 4.268671236691935 MSE test 4.4693376797802395
Epoch 9 / 10 loss: 965.2999305725098


In [38]:
%timeit execute()

MSE train 21.013082131370552 MSE test 20.696800817140645
Epoch 0 / 10 loss: 10134.440954208374
MSE train 12.545776197496407 MSE test 12.337113599790111
Epoch 1 / 10 loss: 3480.995397567749
MSE train 9.447356102936848 MSE test 9.289391673178086
Epoch 2 / 10 loss: 2365.3183670043945
MSE train 7.775859909808188 MSE test 7.654208346241174
Epoch 3 / 10 loss: 1869.3435215950012
MSE train 6.702062047534048 MSE test 6.6122516655822
Epoch 4 / 10 loss: 1576.1149287223816
MSE train 5.946604326796058 MSE test 5.872933607783672
Epoch 5 / 10 loss: 1379.101336479187
MSE train 5.383724419242147 MSE test 5.31846624201368
Epoch 6 / 10 loss: 1236.4591608047485
MSE train 4.940174314430072 MSE test 4.882950850873307
Epoch 7 / 10 loss: 1127.0782809257507
MSE train 4.580763300649352 MSE test 4.528395800531564
Epoch 8 / 10 loss: 1039.6132636070251
MSE train 4.281752188552861 MSE test 4.232313117138328
Epoch 9 / 10 loss: 967.9392924308777
MSE train 4.026561890442507 MSE test 3.9807074758941194
Epoch 0 / 10 los

In [39]:
# pick a user
sample_user = X_test.iloc[99,:]
#get the predicted ratings
sample_user_pred = sess.run(output_layer, feed_dict={input_layer:[sample_user]})